# DevDays 2023: Let's Build! Fetch data from Epic using FHIR API: Challenges and solutions

![image](img/DevDays-01-Cover.jpg)

- This Let's Build! session will delve into the challenges and solutions for integrating a medical application with the Epic FHIR API. Despite being based on the FHIR specification, we found that the Epic FHIR API did not fully support all of its features, making it difficult to discover patients, and encounters, and get incremental resource loading. 
- During the live code session we will implement integration for a simple medical app that fetches and displays laboratory observations for patients in a specific hospital department. We will use Python, Jupyter notebook, and the fhir-py library to implement this integration.
- Alex will face some challenges like limitations in search parameters to get a list of patients and encounters and filter FHIR resources by date. He will consider some alternatives like HL7v2 ADT feed and CDS Hooks that can help to overcome these limitations.
- Keywords: Epic; Python; CDS Hooks

### Clone and play
https://github.com/beda-software/epic_sandbox_devdays_2023

## Context and problem
- Epic FHIR API is patient-centric
- What if we want to build an app for practitioners or to analyze data across different patients

### What are we going to build?
- An app that displays patients that have some red blood cell data
- A source of these data is Epic

### How are we going to build this?
- Use Epic FHIR API (Sandbox) to fetch the data (patients and observations)
- Load data to our FHIR server
- Search data using our FHIR server

## Let's start

### Connect to Epic Sandbox

#### Register an app 

https://fhir.epic.com/Developer/Apps

#### Create a certificate

In [ ]:
# Create a private key
!openssl genrsa -out keys/devdays-sandbox-1.pem 2048

In [ ]:
# Create a public key
!openssl req -new -x509 -key keys/devdays-sandbox-1.pem -out keys/devdays-sandbox-publickey509.pem -subj '/CN=devdays2023'

In [ ]:
# Find the Public Key Certificate Fingerprint
!openssl x509 -noout -fingerprint -sha1 -inform pem -in keys/devdays-sandbox-publickey509.pem

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
EPIC_SANDBOX_BASE_URL = "https://fhir.epic.com/interconnect-fhir-oauth"
FHIR_PATH = "api/FHIR/R4"
EPIC_CLIENT_ID="76596c9b-ec4e-4313-831a-9af4ea0b8b2b"
PRIVATE_KEY_PATH = "keys/devdays-sandbox.pem"

In [ ]:
# Get metadata
import requests

metadata_url = f"{EPIC_SANDBOX_BASE_URL}/{FHIR_PATH}/metadata"
metadata_headers = {"Accept": "application/fhir+json", "Epic-Client-ID": EPIC_CLIENT_ID}
metadata_response = requests.get(metadata_url, headers=metadata_headers)
metadata_all = metadata_response.json()
metadata_auth = metadata_all["rest"][0]["security"]["extension"]

pp.pprint(metadata_auth)

In [ ]:
TOKEN_PATH = "oauth2/token"

#### Auth

In [ ]:
import uuid
import jwt
from datetime import datetime, timedelta, timezone


payload = {
    "iss": EPIC_CLIENT_ID,
    "sub": EPIC_CLIENT_ID,
    "aud": f"{EPIC_SANDBOX_BASE_URL}/{TOKEN_PATH}",
    "jti": str(uuid.uuid4()),
    "iat": datetime.now(timezone.utc),
    "exp": datetime.now(timezone.utc) + timedelta(minutes=1), # max 5 minutes!
}

with open(PRIVATE_KEY_PATH, "rb") as fh:
    signing_key = fh.read()


jwt_token = jwt.encode(payload, signing_key, algorithm="RS384")

pp.pprint(jwt_token)

In [ ]:
auth_token_url = f"{EPIC_SANDBOX_BASE_URL}/{TOKEN_PATH}"

headers = {'Content-Type': 'application/x-www-form-urlencoded'}

data = {
    "grant_type": "client_credentials",
    "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
    "client_assertion": jwt_token,
}

resp = requests.post(auth_token_url, headers=headers, data=data)
print(resp)
token_response_json = resp.json()
pp.pprint(token_response_json)

acces_token = token_response_json["access_token"]

#### Setup an Epic client

In [ ]:
from fhirpy import SyncFHIRClient

epic_client = SyncFHIRClient(
    f"{EPIC_SANDBOX_BASE_URL}/{FHIR_PATH}",
    authorization=f"Bearer {acces_token}",
)

#### Test Epic client

In [ ]:
patient_powell = epic_client.reference('Patient', 'eAB3mDIBBcyUKviyzrxsnAw3').to_resource()
pp.pprint(patient_powell.serialize())

### Set up our FHIR server 

In [ ]:
# Aidbox client
aidbox_url = "https://devdays2023test.aidbox.app"
basic_token = "YmFzaWM6c2VjcmV0"

aidbox_client = SyncFHIRClient(
    f"{aidbox_url}/fhir",
    authorization=f"Basic {basic_token}",
)


In [ ]:
# Test connection to Aidbox
location = aidbox_client.reference('Location', '24e3f3fe-214a-4e71-b0d7-f6f546a3383d').to_resource()
print(location)
# aidbox_metadata = aidbox_client.execute("$metadata", method="GET")
# print(aidbox_metadata)

### Find observations

![image](img/DevDays-02-Background.jpg)

https://fhir.epic.com/Sandbox?api=999

In [ ]:
# LIMITED observation search
labs_all = epic_client.resources("Observation").search(category="laboratory").fetch_all()

In [ ]:
# LIMITED encounter search
encounters_all = epic_client.resources("Encounter").search(date="2021").fetch_all()

In [ ]:
# LIMITED patient search
patient_list = epic_client.resources("Patient").fetch_all()
print(patient_list)

### ADT Feed

[Epic Documentation Interface/HL7v2](https://open.epic.com/Interface/HL7v2)

![image](img/opa-epic.png)

There are many event types

In [ ]:
## ADT Messsage example

adt_message_powell = """
MSH|^~\&|EPIC|EMC|||20150601135823|ADTADM|ADT^A01^ADT_A01|7923
EVN|A01|20150601135823||ADT_EVENT|FS USER^ADT^ADMIN^^^^^^EHS^^^^^EHSMH|20150601135800
PID|eAB3mDIBBcyUKviyzrxsnAw3||E4330777^^^EPIC^MRN~204074777^^^EPI^MR||Does^John^^^^^D||19700601|M|||^^^^^US^L|||||||25234|000-00-0000|||||||||||N
PD1|||EHS HOSPITAL^^10101
PV1|1|EMERGENCY|EMERG^FT13^FT13^^^^^^^^DEPID|EL||||||Emergency||||Non-Healthcr||||||
SELF|||||||||||||||||||||Adm*Conf|||20150601135800
PV2||GENERAL|||||||||||||||||||n|N||||||||||||||||Ambulance
GT1|1|2454|TESTBILL^EDTWORRRRR^^||^^^^^US|||19700601|M|P/F|SLF|000-00-0000
ZG1||||10
"""
print(adt_message_powell)

In [ ]:
import hl7

hl7_message = hl7.parse(adt_message_powell.replace("\n", "\r").strip())
pid_segment = hl7_message.segment("PID")
print(pid_segment)

In [ ]:
patient_id = pid_segment[1]
print(patient_id)

In [ ]:
## Save patient
patient_epic = epic_client.reference("Patient", patient_id).to_resource()
patient_data = patient_epic.serialize()
print(patient_data)

In [ ]:
patient_local = aidbox_client.resource("Patient", id=patient_epic.id, name=patient_data["name"], birthDate=patient_data["birthDate"])
patient_local.save()

### Save all patients

In [ ]:
# Sandbox test data
# https://fhir.epic.com/Documentation?docId=testpatients

# Camila Lopez *
lopez_id = "erXuFYUfucBZaryVksYEcMg3"

# Derrick Lin
lin_id = "eq081-VQEgP8drUUqCWzHfw3"

# Desiree Powell **
powell_id = "eAB3mDIBBcyUKviyzrxsnAw3"

# Elijah Davis
davis_id = "egqBHVfQlt4Bw3XGXoxVxHg3"

# Linda Ross **
ross_id = "eIXesllypH3M9tAA5WdJftQ3"

# Olivia Roberts
roberts_id = "eh2xYHuzl9nkSFVvV3osUHg3"

# Warren McGinnis * **
mcginnis_id = "e0w0LEDCYtfckT6N.CkJKCw3"

sandbox_patient_ids = [lopez_id, lin_id, powell_id, davis_id, ross_id, roberts_id, mcginnis_id]

In [ ]:
for patient_id in sandbox_patient_ids:
    print(f"{'-'*20} {patient_id} {'-'*20}")
    patient_epic = epic_client.reference("Patient", patient_id).to_resource()
    patient_data = patient_epic.serialize()
    print(f"Epic patient: {patient_epic}")
    print(f"Patient data: {patient_data}")
    patient_local = aidbox_client.resource("Patient", id=patient_epic.id, name=patient_data["name"], birthDate=patient_data["birthDate"])
    patient_local.save()
    print(f"Local patient: {patient_local}")

### Save Epic laboratory observations to the local FHIR server

In [ ]:
# getting observations

for patient_id in sandbox_patient_ids:
    print(f"{'-'*20} {patient_id} {'-'*20}")
    lab_list_epic = epic_client.resources("Observation").search(patient=patient_id, category="laboratory").fetch_all()
    print(f"Found labs: {len(lab_list_epic)}")
    for lab in lab_list_epic:
        lab_data = lab.serialize()
        print(f"{'*'*20} {lab.id} {'*'*20}")
        print(lab_data)
            
        lab_local = aidbox_client.resource(
            "Observation", 
            id=lab_data["id"],
            subject=lab_data["subject"],
            category=lab_data["category"], 
            code=lab_data["code"],
            status=lab_data["status"],
            effectiveDateTime=lab_data["effectiveDateTime"]
        )
        if lab_data.get("valueQuantity"):
            lab_local["valueQuantity"] = lab_data.get("valueQuantity")
        if lab_data.get("valueString"):
            lab_local["valueString"] = lab_data.get("valueString")
        
        lab_local.save()

### Find patients with RBC results

In [ ]:
RBC_LOINC_CODE = "789-8"

rbc_results = aidbox_client.resources("Observation").include("Observation", "subject").search(code=RBC_LOINC_CODE).fetch_raw()
print(rbc_results)

## Alternatives to ADT feed

| |ADT feed|CDS Hooks|Patient List|Bulk API|
|-|---------|---------|------------|--------|
|Advantages|Widely supported in Epic|Depends on a use case, prefetch|FHIR API|No special setup from a hospital site|
|Disadvantages|Additional setup required, need to parse|Additional setup required, not specific calls|Additional setup required|Not real-time, likely big amount of data exposed and transferred|

In [ ]:
#### CDS Hooks

Documentation:
- https://cds-hooks.org/specification/current/
- https://open.epic.com/Interface/FHIR#CDSHooks

Hook types:
- patient-view
- order-select
- ...


```
curl
  -X POST \
  -H 'Content-type: application/json' \
  --data @hook-details-see-example-below
  "http://<app_url>/<endpoint-to-handle-hook-call>"

{
  "hookInstance": "d1577c69-dfbe-44ad-ba6d-3e05e953b2ea",
  "fhirServer": "http://hooks.smarthealthit.org:9080",
  "hook": "patient-view",
  "fhirAuthorization": {
    "access_token": "some-opaque-fhir-access-token",
    "token_type": "Bearer",
    "expires_in": 300,
    "scope": "user/Patient.read user/Observation.read",
    "subject": "cds-service4"
  },
  "context": {
    "userId": "Practitioner/example",
    "patientId": "1288992",
    "encounterId": "89284"
  },
  "prefetch": {
    "patientToGreet": {
      "resourceType": "Patient",
      "gender": "male",
      "birthDate": "1925-12-23",
      "id": "1288992",
      "active": true
    }
  }
}
```

In [ ]:
https://fhir.epic.com/Documentation?docId=testpatients

In [ ]:
# Kick-off 
# https://fhir.epic.com/Sandbox?api=10169
# https://hostname/instance/api/FHIR/R4/Group/eIscQb2HmqkT.aPxBKDR1mIj3721CpVk1suC7rlu3yX83/$export

# Accept:application/fhir+json
# Prefer:respond-async

# Status request
# https://fhir.epic.com/Sandbox?api=10170

# https://hostname/instance/api/FHIR/BulkRequest/00000000001755C1C7DB51C0082DA988

## Let's overview what ve have done
- We have found patients with specific observations.
- We have got this data from Epic API
- We have found limitations in Epic FHIR search API
- We used alternatives - ADT feed to overcome these limitations

## Let's discuss
- Questions
- Suggestions
- Ideas

![image](img/DevDays-03-End.jpg)

## Footnotes

### Set up a FHIR Server

There are two options how to setup Aidbox:
- Cloud: https://docs.aidbox.app/overview/aidbox-user-portal
- Local using docker: https://docs.aidbox.app/getting-started/run-aidbox/run-aidbox-locally-with-docker


### Documentation references
- https://fhir.epic.com/
- https://open.epic.com/
- https://github.com/beda-software/fhir-py

### Beda Software
- https://beda.software/